In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

2022-08-30 10:14:22.464495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 10:14:22.464539: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = pd.read_csv('dataset/Toddler Autism dataset July 2018.csv')

In [3]:
data.head()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No
1,2,1,1,0,0,0,1,1,0,0,0,36,4,m,White European,yes,no,family member,Yes
2,3,1,0,0,0,0,0,1,1,0,1,36,4,m,middle eastern,yes,no,family member,Yes
3,4,1,1,1,1,1,1,1,1,1,1,24,10,m,Hispanic,no,no,family member,Yes
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes


In [4]:
data = data.drop(['Case_No', 'Qchat-10-Score', 'Who completed the test'], axis=1)

In [5]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Class/ASD Traits
0,0,0,0,0,0,0,1,1,0,1,28,f,middle eastern,yes,no,No
1,1,1,0,0,0,1,1,0,0,0,36,m,White European,yes,no,Yes
2,1,0,0,0,0,0,1,1,0,1,36,m,middle eastern,yes,no,Yes
3,1,1,1,1,1,1,1,1,1,1,24,m,Hispanic,no,no,Yes
4,1,1,0,1,1,1,1,1,1,1,20,f,White European,no,yes,Yes


In [7]:
encoders = {}
for i in ['Sex', 'Ethnicity', 'Jaundice', 'Family_mem_with_ASD', 'Class/ASD Traits ']:
    encoder = LabelEncoder()
    encoder.fit(data[i])
    print(encoder.classes_)
    data[i] = encoder.transform(data[i])
    encoders[i] = encoder

['f' 'm']
['Hispanic' 'Latino' 'Native Indian' 'Others' 'Pacifica' 'White European'
 'asian' 'black' 'middle eastern' 'mixed' 'south asian']
['no' 'yes']
['no' 'yes']
['No' 'Yes']


In [23]:
print(encoders)

{'Sex': LabelEncoder(), 'Ethnicity': LabelEncoder(), 'Jaundice': LabelEncoder(), 'Family_mem_with_ASD': LabelEncoder(), 'Class/ASD Traits ': LabelEncoder()}


In [8]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Class/ASD Traits
0,0,0,0,0,0,0,1,1,0,1,28,0,8,1,0,0
1,1,1,0,0,0,1,1,0,0,0,36,1,5,1,0,1
2,1,0,0,0,0,0,1,1,0,1,36,1,8,1,0,1
3,1,1,1,1,1,1,1,1,1,1,24,1,0,0,0,1
4,1,1,0,1,1,1,1,1,1,1,20,0,5,0,1,1


In [7]:
#data = data.drop(['Case_No', 'Sex', 'Ethnicity', 'Jaundice', 'Family_mem_with_ASD', 'Who completed the test'], axis=1)

In [9]:
data.columns

Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Age_Mons',
       'Sex', 'Ethnicity', 'Jaundice', 'Family_mem_with_ASD',
       'Class/ASD Traits '],
      dtype='object')

In [10]:
X, y = data.drop(['Class/ASD Traits '], axis=1), data['Class/ASD Traits ']

In [11]:
X.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Sex,Ethnicity,Jaundice,Family_mem_with_ASD
0,0,0,0,0,0,0,1,1,0,1,28,0,8,1,0
1,1,1,0,0,0,1,1,0,0,0,36,1,5,1,0
2,1,0,0,0,0,0,1,1,0,1,36,1,8,1,0
3,1,1,1,1,1,1,1,1,1,1,24,1,0,0,0
4,1,1,0,1,1,1,1,1,1,1,20,0,5,0,1


In [12]:
y

0       0
1       1
2       1
3       1
4       1
       ..
1049    0
1050    1
1051    1
1052    0
1053    1
Name: Class/ASD Traits , Length: 1054, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [14]:
layers = [
    tf.keras.layers.Input((X_train.shape[1],), name="input"),
    tf.keras.layers.Dense(16, activation="relu", name="layer1"),
    tf.keras.layers.Dense(8, activation="relu", name="layer2"),
    tf.keras.layers.Dense(1, activation="sigmoid", name="output")
]

In [15]:
model = tf.keras.Sequential(layers)

2022-08-30 10:14:39.854852: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-30 10:14:39.854916: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-08-30 10:14:39.855907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 16)                256       
                                                                 
 layer2 (Dense)              (None, 8)                 136       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer="Adam", loss="BinaryCrossentropy", metrics=["Accuracy"])

In [18]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
24/24 [==============================] - 1s 1ms/step - loss: 0.9093 - Accuracy: 0.3677
Epoch 2/100
24/24 [==============================] - 0s 1ms/step - loss: 0.6202 - Accuracy: 0.6839
Epoch 3/100
24/24 [==============================] - 0s 2ms/step - loss: 0.5819 - Accuracy: 0.7083
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 0.5489 - Accuracy: 0.7354
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 0.5099 - Accuracy: 0.7395
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 0.4692 - Accuracy: 0.7843
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 0.4108 - Accuracy: 0.7938
Epoch 8/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3533 - Accuracy: 0.8806
Epoch 9/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3017 - Accuracy: 0.9050
Epoch 10/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2797 - Accuracy: 0.8820
Epoch 11/

In [19]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 2ms/step - loss: 0.0595 - Accuracy: 0.9779


[0.05954025685787201, 0.9779179692268372]

In [20]:
y_pred = model.predict(X_test) > 0.5

10/10 [==============================] - 0s 1ms/step


In [21]:
confusion_matrix(y_test, y_pred)

array([[ 89,   6],
       [  1, 221]])

In [22]:
answer_dict_1 = {"Always" : 0, "Usually" : 0, "Sometimes": 1, "Rarely" : 1, "Never" : 1}
answer_dict_2 = {"Always" : 1, "Usually" : 1, "Sometimes": 1, "Rarely" : 0, "Never" : 0}

In [27]:
print("Legal answers: Always, Usually, Sometimes, Rarely or Never")
A = [None for i in range(10)]
A[0] = input("Does your child look at you when you call his/her name?")
A[1] = input("How easy is it for you to get eye contact with your child?")
A[2] = input("Does your child point to indicate that s/he wants something? (e.g. a toy that is out of reach)")
A[3] = input("Does your child point to share interest with you? (e.g. pointing at an interesting sight)")
A[4] = input("Does your child pretend? (e.g. care for dolls, talk on a toy phone)")
A[5] = input("Does your child follow where you're looking? ")
A[6] = input("If you or someone else in the family is visibly upset, does your child show signs of wanting to comfort them? (e.g. stroking hair, hugging them)")
A[7] = input("Would you describe your child's first words as:")
A[8] = input("Does your child use simple gestures? (e.g. wave goodbye)")
A[9] = input("Does your child stare at nothing with no apparent purpose?")
age_mons = int(input("What is your child's age in months?"))
sex = input("Sex of the child(f/m): ")
ethnicity = input("Ethnicity('middle eastern', 'White European', 'Hispanic', 'black', 'asian','south asian', 'Native Indian', 'Others', 'Latino', 'mixed','Pacifica'): ")
jaundice = input("Did the child have jaundice(yes/no): ")
fam_asd = input("Do any of of your family members have ASD(yes/no): ")

Legal answers: Always, Usually, Sometimes, Rarely or Never


In [25]:
encoders['Sex'].classes_

array(['f', 'm'], dtype=object)

In [26]:
encoders['Sex'].transform(np.array(['f']))

array([0])

In [28]:
for i in range(9):
    A[i] = answer_dict_1[A[i]]
A[9] = answer_dict_2[A[9]]

sex = encoders['Sex'].transform(np.array([sex]))
ethnicity = encoders['Ethnicity'].transform(np.array([ethnicity]))
jaundice = encoders['Jaundice'].transform(np.array([jaundice]))
fam_asd = encoders['Family_mem_with_ASD'].transform(np.array([fam_asd]))

In [35]:
sex[0]

0

In [36]:
tmp = A[0],A[1],A[2],A[3],A[4],A[5],A[6],A[7],A[8],A[9], age_mons, sex[0], ethnicity[0], jaundice[0], fam_asd[0]

In [37]:
tmp

(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 34, 0, 4, 1, 0)

In [38]:
input_vals = np.array(tmp).reshape(-1, X.shape[1])

In [39]:
input_vals.shape

(1, 15)

In [40]:
pred = model.predict(input_vals) > 0.5

1/1 [==============================] - 0s 28ms/step


In [41]:
pred

array([[False]])

In [42]:
if(pred):
    print("Your child shows signs of autism")
else:
    print("Your child does not show signs of autism")

Your child does not show signs of autism
